# Sepic Converter [3.3 V, 20 V] -> 10 V
This is the SEPIC converter for converting voltages for the level-shifters of the FET-drivers.
The voltage supply of the SEPIC is recommended to be between [5.5V, 16V]

### Why SEPIC?
- Output only goes above 0 V when the switch is turned on (a.k.a. when sepic starts working)
- Less noise on power line (isolated supply through secondary inductor)
- 10 V level shifting working both at 5 V or at 20 V supply (20 V => 40 V)

## Inductor sizing
We can choose 2 inductors wound around the same common core. This would 
- Give us more boardspace
- Require smaller inductors in terms of magnetization (half as big)

This option seems to be more expensive however, so think about simply buying twice the inductor, since the board-size is not a limitation anyways.

### Variables required for calculation

In [ ]:

#############! DESIGN PARAMETERS ##################
### Take from LTSpice simulation + add on top
I_rms_A = 0.5 # Amps (slightly overdimensioned already)
I_peak_A = 1 # Amps, for each bridge spike of 1 amp 180 degrees shifted

### DIODE Choice:
# 40 V max
# RMS voltage max: 28 V
# 5 A RMS current max
V_diode = 0.55 # SS54
V_max_diode = 40 # 

### MOSFET Choice: AO3400A-mosfet
Vfet_gsth = 1.45 # Volt
Vfet_diode = 1 # Volt
Rfet_dsn = 50e-3 # At 3 amps, 2.5 V
Vfet_dsn = Rfet_dsn * I_rms_A
print(f"Vfet_dsn {Vfet_dsn}")

# Required Output Voltage
Vout = 10

Vfet_dsn 0.025


### Calculations for (Vin_min = 5, Vin_max = 20V)

In [18]:

#! >>> SET INPUT VOLTAGE min max
Vin_list = [20, 5]
#! <<< SET INPUT VOLTAGE

In [19]:
for Vin in Vin_list:
	print(f"\r\n##################################################")
	print(f"################# VIN == {Vin} ###################")
	print(f"##################################################")

	D_sepic = (Vout + V_diode) / (Vout + Vin - Rfet_dsn * I_rms_A  + V_diode)
	print(f"Duty cycle: {D_sepic}, time the switch is closed")

	if (D_sepic > 0.5):
		print(f"Warning slope compensation required")

	##### MINIMUM INDUCTOR SIZE #######
	print(f"\r\nMINIMUM INDUCTOR SIZE")
	freq_sw = 500e3

	I1_sepic = D_sepic * I_rms_A / (1-D_sepic)
	I2_sepic = I_rms_A

	# Inductance
	L1_sepic = (Vin - Vfet_dsn) * (1-D_sepic) / (2 * I_rms_A * freq_sw)
	L2_sepic = (Vin - Vfet_dsn) * (D_sepic) / (2 * I_rms_A * freq_sw)

	print(f"L1_sepic: {L1_sepic*1e6} uH, L2_sepic: {L2_sepic*1e6} uH")


##################################################
################# VIN == 20 ###################
##################################################
Duty cycle: 0.3456183456183456, time the switch is closed

MINIMUM INDUCTOR SIZE
L1_sepic: 26.142547092547094 uH, L2_sepic: 13.807452907452907 uH

##################################################
################# VIN == 5 ###################
##################################################
Duty cycle: 0.679549114331723, time the switch is closed
Warning slope compensation required

MINIMUM INDUCTOR SIZE
L1_sepic: 3.1884863123993554 uH, L2_sepic: 6.761513687600644 uH


### SEPIC passives sizing
####  Setting R_sense
Increasing $R_{sense}$ makes you reach the COMP-pin threshold sooner.
- Lower peak current (reduced output power)
- More conservative operation
- Increased inner-loop bandwidth (smoother control)

Decreasing $R_{sense}$
- Higher $I_{L}$, peak-current

NOTE:
- Changing R_sense affects the effect of Rsl-compensation
	- You may need more / less Rsl to stabilize at D>0.5
	- e.g.: reducing the Rsense, decreases the ramp which requires more Rsl voor stabilization
#### Required slope compensation 
Check **[dcdc_choice.md](../dcdc_choice.md)** for more information about current sensing slope.

- Existing CCM: $$I_{Comparator} = G_{m, cs} * I_{L} * R_{Sense}$$
	- This is proportional to the inductor current
- Existing slope compensation (about 100 mV), enough to stabilize to D=0.55
	- 100 mV -> 100 mV / 40 uA = 2.5 kOhm of compensation
- Slope-compensation $$I_{Comparator} = G_{m, cs} * I_{L} * R_{Sense} + R_{sl} * 40 uA$$


In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
#! >>> INSERT CHOICE
# Set ESR of output capacitor
ESR_cout = 0.1 # ESR of output capacitor
L1_ch = 100e-6#L1_sepic
L2_ch = 100e-6#L2_sepic
R_sl = 2.5e3 # Slope compensation resistor
#! <<< INSERT CHOICE
import math 
for Vin in Vin_list:
	print(f"\r\n##################################################")
	print(f"################### VIN == {Vin} #####################")
	print(f"##################################################")

	D_sepic = (Vout + V_diode) / (Vout + Vin - Rfet_dsn * I_rms_A  + V_diode)
	print(f"Duty cycle: {D_sepic}, time the switch is closed")

	if (D_sepic > 0.5):
		print(f"Warning slope compensation required")

	##### MINIMUM INDUCTOR SIZE #######
	print(f"\r\nMINIMUM INDUCTOR SIZE")
	freq_sw = 500e3

	I1_sepic = D_sepic * I_rms_A / (1-D_sepic)
	I2_sepic = I_rms_A

	# Inductance
	L1_sepic = (Vin - Vfet_dsn) * (1-D_sepic) / (2 * I_rms_A * freq_sw)
	L2_sepic = (Vin - Vfet_dsn) * (D_sepic) / (2 * I_rms_A * freq_sw)

	print(f"L1_sepic: {L1_sepic*1e6} uH, L2_sepic: {L2_sepic*1e6} uH")
	
	print(f"\r\nINDUCTOR CHOICE")
	print(f"L1_ch: {L1_ch*1e6} uH, L2_ch: {L2_ch*1e6} uH")

	##### PEAK CURRENT CALCULATION #######
	print(f"\r\n#### PEAK CURRENT CALCULATION ####")
	# Peak current
	dIl1 = (Vin - Vfet_dsn) * D_sepic / (L1_ch * freq_sw)
	dIl2 = (Vin - Vfet_dsn) * D_sepic / (L2_ch * freq_sw)
	print(f"dIl1: {dIl1} A, dIl2: {dIl2} A")

	Il1_peak = I1_sepic + dIl1 / 2
	Il2_peak = I2_sepic + dIl2 / 2
	print(f"Il1_peak: {Il1_peak} A, Il2_peak: {Il2_peak} A")

	######## OUTPUT VOLTAGE RIPPLE #########
	print(f"\r\n#### OUTPUT VOLTAGE RIPPLE (ESR-DEPENDENT) ####")

	dVout = ((I_rms_A / (1-D_sepic)) + dIl2 / 2) * ESR_cout
	print(f"dVout: {dVout} V")

	######## SEPIC CAPACITOR SELECTION #########
	print(f"\r\n#### SEPIC CAPACITOR SELECTION ####")
	I_sw_peak = I1_sepic + I_rms_A + (dIl1 + dIl2) / 2
	I_swrms = math.sqrt((I_sw_peak**2 - I_sw_peak*(dIl1 + dIl2) + (dIl1 + dIl2)**2 / 3)*D_sepic)
	print(f"I_SW_RMS: {I_swrms} A")
	print(f"I_SW_PEAK: {I_sw_peak} A")

	I_cap_rms = math.sqrt(I_swrms**2 + (Il1_peak**2-Il1_peak*dIl1+dIl1**2) * (1-D_sepic))
	V_cap_rms = Vin
	C_min = L1_ch * I_rms_A**2 / (Vin-Vfet_dsn)**2

	print(f"[C_coupling_minimum] {C_min*1e6} uF, I_cap_rms {I_cap_rms} A, V_max: {V_cap_rms} V")

	######## INPUT CAPACITOR SELECTION #########
	print(f"\r\n#### INPUT CAPACITOR SELECTION ####")
	I_cin_rms = (D_sepic / (2 * math.sqrt(3))) * ((Vin - Vfet_dsn) / (L1_ch * freq_sw))
	print(f"Input capacitor RMS current {I_cin_rms} A")

	######## OUTPUT CAPACITOR SELECTION #########
	print(f"\r\n#### OUTPUT CAPACITOR SELECTION ####")
	I_cout_rms = math.sqrt((I_sw_peak**2 - I_sw_peak * (dIl1 + dIl2) + ((dIl2+ dIl1)**2)/3.0) * (1-D_sepic) - I_rms_A**2)
	print(f"output capacitor RMS current: {I_cout_rms} A")


	######## SLOPE COMPENSATION #######
	print(f"\r\n#### SLOPE COMPENSATION | SENSE RESISTOR SELECTION ####")
	V_sense = 0.156 # Volt - FIXED current sense threshold voltage
	V_sl = 92e-3 # Internal compensation ramp voltage
	V_sl_ratio = 0.49
	K_sl = 40e-6
	dV_sl = K_sl * R_sl

	### R_sen estimation (boost equations)
	R_sense = (V_sense - D_sepic * (V_sl + dV_sl)) / I_sw_peak # (eq. 48)
	print(f"R_sense: {R_sense} Ohm")
	print(f"I_max: {V_sense / R_sense} A")
	### Slope compensation
	# R_sl = ((R_sense * (Vout - 2*Vin)) / (2*freq_sw*L1_ch) - V_sl) / K_sl
	print(f"Slope compensation resistor: {R_sl}, Duty: {D_sepic}")


##################################################
################### VIN == 20 #####################
##################################################
Duty cycle: 0.3456183456183456, time the switch is closed

MINIMUM INDUCTOR SIZE
L1_sepic: 26.142547092547094 uH, L2_sepic: 13.807452907452907 uH

INDUCTOR CHOICE
L1_ch: 100.0 uH, L2_ch: 100.0 uH

#### PEAK CURRENT CALCULATION ####
dIl1: 0.13807452907452908 A, dIl2: 0.13807452907452908 A
Il1_peak: 0.333117364662421 A, Il2_peak: 0.5690372645372646 A

#### OUTPUT VOLTAGE RIPPLE (ESR-DEPENDENT) ####
dVout: 0.0833117364662421 V

#### SEPIC CAPACITOR SELECTION ####
I_SW_RMS: 0.45163558876069376 A
I_SW_PEAK: 0.9021546291996856 A
[C_coupling_minimum] 0.06265654345779532 uF, I_cap_rms 0.5088878069613065 A, V_max: 20 V

#### INPUT CAPACITOR SELECTION ####
Input capacitor RMS current 0.03985868326470508 A

#### OUTPUT CAPACITOR SELECTION ####
output capacitor RMS current: 0.3690508811782395 A

#### SLOPE COMPENSATION | SENSE RESISTOR SELECTION

## Ceramic capacitance power dissipation
Source: https://forum.digikey.com/t/calculating-capacitor-esr-from-tan/2633

In [25]:

#! >>> ENTER CAPACITOR DETAILS
coupling_cap = 4.7e-6
tan_delta = 3.5e-2 # < 3.5e-2 according to datasheet
#! <<<

ceramic_ESR=tan_delta/(2*math.pi * freq_sw * coupling_cap)
print(f"ESR: {ceramic_ESR}, I_cap_rms: {I_cap_rms}")
P_coupling = ceramic_ESR*I_cap_rms**2
print(f"Coupling power: {P_coupling} W")

ESR: 0.0023703927694537605, I_cap_rms: 1.420136572499947
Coupling power: 0.004780579419063763 W


# Rsense selection
R_sense has to be the smallest possible value: 50 mohm

In [32]:
R_sense = 50e-3
P_sense = 333e-3 # Chosen
I_max_rms = math.sqrt(P_sense/R_sense)
print(f"Resistor can withstand: {I_max_rms} A rms")

Resistor can withstand: 2.5806975801127883 A rms


## Setting output voltage
- Don't forget to connect an output capacitor between feedback and ground pins
	- Reduces bandwidth
	- Reduces susceptibility to noise

In [21]:
RF1 = 60e3 # 60 kOhms feedback in example
RF2 = (1.26 * RF1) / (Vout - 1.26)

print(f"Resistors: {RF1*1e-3:.2f} kOhm, {RF2*1e-3:.2f} kOhm")

Resistors: 60.00 kOhm, 8.65 kOhm


## Setting Current Limit

### Compensation design for RHP in voltage feedback loop
(Check TI-AN1484)
The lowest cut-off frequency appears at the highest duty cycle (lowest voltage), so calculate all of this for the 5 V input voltage
The right-half plane zero:
- Shifts the phase -90 degrees, so make sure crossover happens way before f_rhzp
- Is lowest and thus more critical with D highest, so choose V = 5 V (lowest) since D is highest there.

The resonant LC pole:
- Make sure the bandwidth is far below that, at resonance we get a phase-shift of -180 degrees

Crossover frequency:
- Frequency where open-loop transfer function drops below 0 dB (Gain: 1)

Make sure to lower cutoff frequency so positive feedback is avoided.

In [22]:

#! >>> INSERT COUPLING CAPACITOR CHOICE HERE
C_coupling_ch = 4.7e-6 # F
D_sepic_max = 0.65
Vin = 5
Cin_ch = 100e-6
Cout_ch = 100e-6
#! <<< INSERT COUPLING CAPACITOR CHOICE HERE

# LOAD POLE ESTIMATION
print(f"\r\n##################################################")
print(f"################### VIN == {Vin} #####################")
print(f"##################################################")
# RIGHT HALF PLANE ZERO
f_RHPZ = (1 - D_sepic_max) ** 2 * Vout / (2 * math.pi * D_sepic_max * L2_ch * 0.5 * I_rms_A)
print(f"Right Half plane zero: {f_RHPZ*1e-3:.2f}kHz")
# RESONANT LC POLE 
f_R = 1 / (2*math.pi * math.sqrt(L2_ch * C_coupling_ch))
print(f"Cs-L2 Resonance: {f_R*1e-3:.2f}kHz")
f_crossover = min(f_R, f_RHPZ) / 6
print(f"Crossover frequency: {f_crossover*1e-3:.2f}kHz")

#### COMPENSATION NETWORK CHOICE
# CHOICE OF RC
Gcs = 1/R_sense # Current sense gain
print(f"Gcs: {Gcs}")
Gma = 800e-6 # Error Amplifier Transconductance
Rc = (2 * math.pi * f_crossover * Cout_ch * Vout**2 * (1+D_sepic_max))
Rc = Rc / (Gcs*Gma*1.26*Vin*D_sepic_max)
print(f"Rc: {Rc*1e-3:.2f}kOhm")

# CHOICE OF CC1 (1/4th crossover frequency)
Cc1 = 4 / (2 * math.pi * f_crossover * Rc)
Cc2 = Cout_ch * ESR_cout / Rc
print(f"Cc1: {Cc1*1e9}nF, Cc2: {Cc2*1e9}nF")


##################################################
################### VIN == 5 #####################
##################################################
Right Half plane zero: 12.00kHz
Cs-L2 Resonance: 7.34kHz
Crossover frequency: 1.22kHz
Gcs: 17.419387367182225
Rc: 2.22kOhm
Cc1: 234.07399956652642nF, Cc2: 4.4987618664220905nF


## Mosfet / Diode Selection

#### Using the AO3400A
It seems to be better to select a MOSFET like AO3400A, since it has a much lower Vgs(th) and thus has a much larger range at which it can supply power to the processing-board.
- Input voltage will be max 20 V
- Output voltage will be max 9-10 V
- Vgs will be max 7 V (limit is 12 V)

**Stats:**
- Vgs(th): 1-2 V
- Id_max: 4-5 A
- Pd: 1.4 W
- Vgs: 12 V max
- Vds_max: 30 V
- Rdson: 30 mOhm

#### Finding an LTSpice Model
Use the IRLML6344PbF FET model for the SEPIC instead of the AO3400A.

**Stats:**
- Vgs(th): 0.5-1.1 V
- Id_max: 7 A
- Pd: 1.2 W
- Vgs: 12 V max
- Vds_max: 20 V
- Rds_on: 30 mOhm

In [228]:
V_sw_peak = max(Vin_list) + Vout + V_diode
print(f"V_sw_peak: {V_sw_peak}")

V_sw_peak: 30.55


# LTSpice Simulation
![](sepic_fetdriver_circuit.png)

### Things to check
- Is the current sense resistor well-sized?
	- If too small: overcurrent protection might kick in (at 230 mV)
	- If too small: Vsl might kick in too early (92 mV at 5 V), and the switching will occur as a consequence.
- Is the Rsl correct?
	- Too large: can't reach high enough
	- Too small: bad control
- Can your SEPIC even reach the output voltage and current required for that duty cycle?
	- There is a theoretical and a practical max possible duty cycle.
	- D_max + parasitics might bring the duty cycle down to a max of 0.6-0.7 + losses in inductor.
	- You can only deliver a voltage so high with a given SEPIC.
- Make sure your load resistance actually represents the current draw from the source.
- Do you have any weird transients
	- Add line / parasitic inductance / resistance for a more realistic simulation

## 20 V input

![](sepic_fetdriver_20Vin.png)

### Soft Start (0->4 ms)
- Error-amplifier charges its soft-start capacitor.
- Internal soft-start delay is 4 ms.